## 5. Analisador de Extremos Climáticos e Mudanças Climáticas
Crie um sistema robusto para análise de eventos extremos:

- Implemente análise de valores extremos usando distribuições GEV (Generalized Extreme Value)
- Calcule períodos de retorno para eventos como chuvas intensas, temperaturas extremas
- Analise tendências climáticas usando testes de Mann-Kendall e regressão Sen's slope
- Gere projeções de cenários futuros, mapas de risco e relatórios de adaptação climática

## Dependencias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import genextreme, norm
import warnings
warnings.filterwarnings('ignore')

## Mann Kendel - Biblioteca para analise climatica

In [3]:
try:
    from pymannkendall import original_test as mk_test
except:
    print("Instalando pymannkendall: pip install pymannkendall")
    # Implementação manual do teste Mann-Kendall
    def mk_test(data):
        n= len(data)
        s = 0
        for i in range(n-1):
            for j in range(i+1, n):
                if data[j]> data[i]:
                    s += 1
                elif data[j]< data[i]:
                    s -= 1
        
        var_s = n*(n-1)*(2*n+5)/18
        if s> 0:
            z = (s-1)/np.sqrt(var_s)
        elif s< 0:
            z = (s+1)/np.sqrt(var_s)
        else:
            z = 0
        p_value = 2*(1-stats.norm.cdf(abs(z)))
        trend = 'increasing' if z > 0 else 'decreasing' if z < 0 else 'no trend'

        class MKResult:
            def __init__(self, trend, p, z, s):
                self.trend = trend
                self.p = p
                self.z = z
                self.s = s
        return MKResult(trend, p_value, z, s)


## Classe Analisador de Eventos Climáticos

In [4]:
class ClimateExtremesAnalyzer:
    """
    Classe para analisar os dados de clima extremos.
    """
    def __init__(self):
        self.data = None
        self.extreme_data = None
        self.gev_params = {}
        self.trends = {}
    
    def load_data(self, data_path=None, data_df=None):
        """
        Carrega dados climáticos
        """
        if data_df is not None:
            self.data = data_df
        elif data_path:
            # carregar de arquivo
            if data_path.endswith('.csv'):
                self.data = pd.read_csv(data_path, parse_dates=['date'])
            else:
                raise ValueError("Formato de arquivo não suportado")
        else:
            # Gerar dados sinteticos para demonstração
            self.generate_synthetic_data()
        
        print(f"Dados carregados: {len(self.data)} registros")
        return self.data
    
    def generate_synthetic_data(self):
        """
        Gera dados climáticos sintéticos para demonstração
        """
        np.random.seed(42)

        # Gerar 30 anos de dados diarios
        dates = pd.date_range('1985-01-01', '2024-12-31', freq='D')
        n_days = len(dates)

        # Tendencia de aquecimento
        trend = np.linspace(0, 2, n_days) # aumento de 2°C em 30 anos

        # Sazonalidade
        day_of_year = dates.dayofyear
        seasonal = 10 * np.sin(2*np.pi*day_of_year / 365.25)

        # Temperatura base + tendencia + sozonalidade + ruido
        temperature = 20 + trend + seasonal + np.random.normal(0,3, n_days)

        #Eventos extremos ocasionais
        extreme_events = np.random.exponential(0.1, n_days) * np.random.choice([0,1], n_days, p=[0.95, 0.05])
        temperature += extreme_events * 15

        # Precipitação (distribuição gamma)
        precipitation = np.random.gamma(0.5, 2, n_days)
        # Eventos extremos de chuva
        extreme_rain = np.random.choice([0,1], n_days, p=[0.98, 0.02]) * np.random.exponential(50,n_days)
        precipitation += extreme_rain

        self.data = pd.DataFrame({
            'date': dates,
            'temperature': temperature,
            'precipitation': precipitation,
            'year': dates.year,
            'month': dates.month
        })

        print("Dados sintéticos gerados com sucesso!")
    
    def extract_extremes(self, variable='temperature', method='block_maxima', block_size='year'):
        """
        Extrai valores extremos usando diferentes métodos
        """
        if method == 'block_maxima':
            if block_size == 'year':
                extremes = self.data.groupby('year')[variable].max()
            elif block_size == 'month':
                extremes = self.data.groupby(['year', 'month'])[variable].max
            else:
                raise ValueError("Block size deve ser year ou month")
        
        elif method == 'peaks_over_threshold':
            # Método POT (Peaks Over Threshold)
            threshold = np.percentile(self.data[variable], 95) # Top 5%
            extremes = self.data[self.data[variable] > threshold][variable]
        
        self.extreme_data = extremes
        print(f"Extraidos {len(extremes)} valores extremos usando método {method}")
        return extremes
    
    def fit_gev_distribution(self, variable='temperature'):
        """
        Ajusta distribuição GEV (Generalized Extreme Value) aos dados extremos
        """

        if self.extreme_data is None:
            self.extract_extremes(variable)
        
        # Ajustar distribuição GEV
        gev_params = genextreme.fit(self.extreme_data)

        self.gev_params[variable] = {
            'shape': gev_params[0], # parâmetro de forma (xi)
            'location': gev_params[1], # parâmetro de localização (mu)
            'scale': gev_params[2]  # parâmetro de escala (sigma)
        }

        # Teste de ajuste Kolgomorov-Sirnov
        ks_stat, ks_p_value = stats.kstest(self.extreme_data, lambda x: genextreme.cdf(x, *gev_params))

        print(f"Parametros GEV para {variable}: ")
        print(f"  Forma (ξ): {gev_params[0]:.4f}")
        print(f"  Localização (μ): {gev_params[1]:.4f}")
        print(f"  Escala (σ): {gev_params[2]:.4f}")
        print(f"  Teste KS: estatística = {ks_stat:.4f}, p-valor = {ks_p_value:.4f}")

        return gev_params
        
    
    def celculate_return_periods(self, variable='temepature', return_periods=[2,5,10,25,50,100]):
        """
        Calcula períodos de retorno para diferentes níveis
        """
        if variable not in self.gev_params:
            self.fit_gev_distribution(variable)
        
        params = self.gev_params[variable]
        shape, loc, scale = params['shape'], params['location'], params['scale']

        return_levels = {}

        for T in return_periods:
            # Formula para periodo de retorno usando GEV
            if abs(shape) < 1e-6: # Gumbel (shape ≈ 0)
                level = loc - scale * np.lo(-np.log(1 - 1/T))
            else: # GEV Geral
                level = loc + (scale/shape) * ((-np.log(1 - 1/T))**(-shape) - 1)
            
            return_levels[T] = level

        return return_levels

    def mann_kendall_test(self, variable='temperature', alpha=0.05):
        """
        Realiza teste de Mann-Kendall para detectar tendências
        """
        annual_data = self.data.groupby('year')[variable].mean()

        # Teste Mann-Kendall
        mk_result = mk_test(annual_data.values)

        # Sen's slope (estimativa robusta da inclnação
        n = len(annual_data)   
        slopes = []

        for i in range(n-1):
            for j in range(i+1, n):
                slope = (annual_data.iloc[j] - annual_data.iloc[i]) / (j-1)
                slopes.append(slope)

        sens_slope = np.median(slopes)

        self.trens[variable] = {
            'trend': mk_result.trend,
            'p_value': mk_result.p,
            'z_score': mk_result.z,
            'sens_slope': sens_slope,
            'significant': mk_result.p < alpha
        }

        print(f"Anáise de tendencia para {variable}: ")
        print(f" Tendencia: {mk_result.trend}")
        print(f" Sen's slope: {sens_slope:.6f} por ano")
        print(f" Z-score: {mk_result.z:.4f}")
        print(f" P-valor: {mk_result.p:.6f}")
        print(f" Significativo (α={alpha}): {mk_result.p < alpha}")

        return self.trens[variable]
    
    def project_future_scenarios(self, variable='temperature', future_years=30, scenarios=None):
        """
        Gera projeções de cenários futuros baseadas em tendências
        """
        if scenarios is None:
            scenarios = {
                'Baixo': 0.5, # Fator de multiplicação da tendência atual
                'Médio': 1.0, # Tendência atual mantida
                'Alto': 1.5 # Tendência acelerada
            }
        
        if variable not in self.trends:
            self.mann_kendall_test(variable)
        
        current_trend = self.trends[variable]['sens_slope']
        current_year = self.data['year'].max()
        future_years_range = range(current_year + 1, current_year + future_years + 1)

        # Valor base (média dos ultimos 5 anos)
        base_value = self.data[self.data['year'] >= current_year - 4].groupby('year')[variable].mean().mean()

        projections = {}

        for scenario_name, factor in scenarios.items():
            yearly_change = current_trend * factor
            projection = []

            for i, year in enumerate(future_years_range):
                projected_value = base_value + (yearly_change * (i + 1))
                projection.append(projected_value)

            projections[scenario_name] = {
                'years': list(future_years_range),
                'values': projection
            }
        
        return projections
    
    def generate_risk_assessment(self, variable='temperature'):
        """
        Gera avaliação de risco baseada em extremos e tendências
        """
        # Calcular periodos de retorno
        return_levels = self.celculate_return_periods(variable)

        # Analise de tendencia
        if variable not in self.trends:
            self.mann_kendall_test(variable)
        
        trend_info = self.trends[variable]

        # Classsificação de risco
        if trend_info['significant'] > 0:
            if trend_info['sens_slope'] > 0:
                trend_risk = "Alto" if abs(trend_info['sens_slope']) > 0.1 else "Médio"
            else:
                trend_risk = "Baixo"
        else:
            trend_risk = "Baixo"
        
        # Risco de extremos basado na variabilidade
        current_std = self.data.groupby('year')[variable].mean().std()
        if current_std > self.data[variable].std() * 0.8:
            extreme_risk = "Alto"
        elif current_std > self.data[variable].std() * 0.5:
            extreme_risk = "Médio"
        else:
            extreme_risk = "Baixo"
        
        risk_assesment = {
            'variable': variable,
            'trend_risk': trend_risk,
            'extreme_risk': extreme_risk,
            'return_levels': return_levels,
            'trend_info': trend_info,
            'recommendations': self._generate_recommendations(trend_risk, extreme_risk, variable)
        }

        return risk_assesment
    
    def _generate_recommendations(self, trend_risk, extreme_risk, variable):
        """
        Gera recomendações baseadas no nível de risco
        """
        recommendations = []

        if variable == 'temperature':
            if trend_risk == "Alto":
                recommendations.extend([
                    "Implementar sistemas de resfriamento eficientes",
                    "Desenvolver planos de adaptação ao calor extremo",
                    "Monitorar impactos na saúde pública"
                ])
            elif variable == 'precipitation':
                if trend_risk == "Alto":
                    recommendations.extend([
                        "Melhorar sistemas de drenagem urbana",
                        "Implementar infraestrutura verde",
                        "Desenvolver reservatórios de contenção"
                    ])
                if extreme_risk == "Alto":
                    recommendations.extend([
                        "Criar sistemas de alerta de enchentes",
                        "Estabelecer rotas de evacuação",
                        "Implementar seguros contra desastres naturais"
                    ])
            
            if not recommendations:
                recommendations.append("Manter monitoramento continuo das condições climáticas")

        return recommendations



